In [102]:
from additional_func import get_and_prep_func, train_test_split, create_torch_dataloader, accuracy
from classes import NeuralNet,AdaBoost

### Getting prepared dataset

In [103]:
df = get_and_prep_func()

In [104]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,log(t_90)
0,-0.062049,-0.111054,0.110340,-0.362397,1.049038,0.509581,-0.910055,0.601275,-1.683499,1.102286,1.0
1,-0.030799,0.054564,0.312363,0.257292,-1.020113,-0.080256,-0.133488,2.007042,0.240627,0.526222,1.0
2,0.014842,0.059207,0.326520,0.616788,-0.978302,-0.175741,-0.138935,1.205231,1.247597,0.325159,1.0
3,-0.041400,-1.177580,-0.350045,0.836391,0.180635,0.368622,2.074885,0.460932,1.857980,-1.629638,1.0
4,-0.035763,0.230788,0.286167,0.091809,-0.810984,-0.172459,-0.272057,1.456883,0.836236,0.767478,1.0


### Train test split

In [105]:
df_train, df_test = train_test_split(df)

### Creating DataLoader

In [106]:
train_batch_size = 50
test_batch_size = 50

In [107]:
train_dataloader = create_torch_dataloader(
    df_train, train_batch_size, shuffle=True)
test_dataloader = create_torch_dataloader(
    df_test, test_batch_size, shuffle=True)

# Model 1: Simple neural network with one hidden layer

### Hyperparameters

In [108]:
learning_rate = 1e-3
n_epoch = 500
size_of_hidden_layer = 20

### Initializing model

In [109]:
model1 = NeuralNet(10,size_of_hidden_layer)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=learning_rate)
model1

NeuralNet(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

### Training model

In [110]:
n_total_steps = len(train_dataloader)
test_acc_mean = np.array([])
test_acc_std = np.array([])
step = np.array([0])
for epoch in range(n_epoch):
    for i, (X, labels) in enumerate(train_dataloader):
        pred = model1(X)
        loss = criterion(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch) % 100 == 0 and i == 0:
            acc_mean, acc_std = accuracy(model1, test_dataloader)
            test_acc_mean = np.append(test_acc_mean, acc_mean)
            test_acc_std = np.append(test_acc_std, acc_std)
            step = np.append(step, step[-1] + 10)
            print(
                f'Epoch {epoch}/{n_epoch}, Step {i}/{n_total_steps}, train_loss: {loss:.6f}, accuracy on test: {acc_mean:.1f}, std: {acc_std:.1f}')


Epoch 0/500, Step 0/42, train_loss: 0.706668, accuracy on test: 61.9, std: 9.7
Epoch 100/500, Step 0/42, train_loss: 0.049435, accuracy on test: 92.9, std: 3.8
Epoch 200/500, Step 0/42, train_loss: 0.090635, accuracy on test: 92.9, std: 4.1
Epoch 300/500, Step 0/42, train_loss: 0.149682, accuracy on test: 92.2, std: 3.0
Epoch 400/500, Step 0/42, train_loss: 0.093578, accuracy on test: 92.2, std: 3.0


# Model 2: AdaBoost with simple neural network as weak classifier

### Hyperparameters

In [101]:
learning_rate = 1e-3
n_steps = 500
size_of_hidden_layer = 20
T = 10

### Initializing model

In [98]:
Ada = AdaBoost(weak_classifier=NeuralNet, T = T, df_train=df_train)

### Training model

In [99]:
Ada.main_train(layer_length=size_of_hidden_layer, n_steps = n_steps, lr = learning_rate)

In [100]:
final_acc, final_std=accuracy(Ada.predict, test_dataloader)
print(f'Final accuracy: {final_acc:.3f} std: {final_std:.3f} ')

Final accuracy: 93.123 std: 4.702 


# Results

There are no significant differences in the models